In [1]:
from __future__ import print_function, division
from glob import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

import copy

plt.ion()   # interactive mode
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

!pip install efficientnet_pytorch

In [2]:
data_transforms = {
    'Train': transforms.Compose([
                                 transforms.RandomResizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                 ]),
    'Validation': transforms.Compose([
                              transforms.Resize(256),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                              ]),
    'Testing': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# data_dir  = 'drive/MyDrive/MyThesis/hymenoptera_data/*'
data_dir = 'drive/MyDrive/MyThesis/oxford/Master'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['Train', 'Validation','Testing']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['Train', 'Validation','Testing']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Validation','Testing']}

class_names = image_datasets['Train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
net = torch.load('drive/MyDrive/MyThesis/resnet50oxford.model')
net2 = torch.load('drive/MyDrive/MyThesis/efb750oxford.model')

net.to(device)
net2.to(device)

In [4]:
class_correct = list(0. for i in range(37))
class_total = list(0. for i in range(37))

def onebyoneacc(net):
    hasil = []
    array_sorting = []
    with torch.no_grad():
        for data in dataloaders['Testing']:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(2):
              # print(i)
              # print(labels[i])
              label = labels[i]
              class_correct[label] += c[i].item()
              class_total[label] += 1

   
    for i in range(37):
        mantul = (100 * class_correct[i] / class_total[i])
        array_sorting.append({"class": class_names[i], "akurasi": round(mantul,2)})
        # print(f'Akurasi dari {class_names[i]}: {round(mantul,2)}')
        # print('Accuracy of %5s : %2d %%' % (class_names[i], 100 * class_correct[i] / class_total[i]))
    return array_sorting

In [16]:
result_1 = onebyoneacc(net)
result_1.sort(key = lambda json: json['akurasi'], reverse=True)

result_2 = onebyoneacc(net2)
result_2.sort(key = lambda json: json['akurasi'], reverse=True)


In [22]:
# importing pandas as pd 
import pandas as pd 
df_1 = pd.DataFrame(result_1)
print("=======================================")
print("                ResNet50")
print("=======================================")
print(df_1)

                ResNet50
                         class  akurasi
0                       Sphynx   100.00
1                       bombay   100.00
2                      siamese   100.00
3            yorkshire_terrier   100.00
4               great_pyrenees    96.00
5                japanese_chin    95.83
6                   pomeranian    95.24
7                    chihuahua    92.59
8                 egyptian_mau    92.31
9           miniature_pinscher    91.67
10                newfoundland    91.30
11                         pug    91.30
12                basset_hound    91.18
13                    keeshond    90.00
14                     persian    89.47
15          german_shorthaired    88.00
16                  leonberger    86.96
17            american_bulldog    84.62
18               saint_bernard    84.62
19  staffordshire_bull_terrier    84.62
20                  Abyssinian    82.14
21                Russian_Blue    82.14
22                     samoyed    82.14
23           br

In [19]:
# importing pandas as pd 
import pandas as pd 
df_2 = pd.DataFrame(result_2)
print("=======================================")
print("           EfficienNet-b7")
print("=======================================")
print(df_2)

           EfficienNet-b7
                         class  akurasi
0                      siamese   100.00
1            yorkshire_terrier   100.00
2                       Sphynx    97.06
3                 egyptian_mau    94.59
4                       bombay    93.94
5                 newfoundland    93.94
6           miniature_pinscher    93.55
7               great_pyrenees    93.33
8                          pug    93.10
9                 basset_hound    92.86
10                   chihuahua    91.67
11               japanese_chin    90.91
12                  leonberger    90.91
13          german_shorthaired    90.62
14                  pomeranian    90.00
15                    keeshond    88.89
16  staffordshire_bull_terrier    88.57
17                     persian    88.00
18           british_shorthair    87.50
19                     samoyed    87.50
20                  Abyssinian    86.11
21            scottish_terrier    86.11
22                Russian_Blue    83.78
23            